# Dense Cholesky Decomposition: By Row
For a dense SPD A a simple row oriented in-place Cholesky decomposition is naturally organized to give the Upper triangular factor U someting like this. 

In [10]:
function DenseCholRow(U) 
    m = size(U)[1]
    for k in 1:m
        for j in k+1:m
            U[j,j:m] -= (U[k,j]/U[k,k])*U[k,j:m] 
        end
        U[k,k:m] = U[k,k:m]/sqrt(U[k,k])
    end
    triu!(U)
    end

DenseCholRow (generic function with 1 method)

# Dense Cholesky Decomposition: By Column
For a dense SPD A a simple column oriented in-place Cholesky decomposition can be naturally organized to give the lower triangular matrix L.

In [22]:
dot(A[1,1:2],A[1:2,2])

1.1697869728408854

In [58]:
function DenseCholCol1(A)
    R = copy(A)
    n = size(R)[1]
    for j in 1:n
        for i in 1:j-1
            # println("i ", i)
            R[i,j] = (A[i,j] - dot(R[1:i-1,i],R[1:i-1,j]))/R[i,i]  
        end
        # println("j ", j)
        R[j,j] = sqrt(A[j,j] - dot(R[1:j-1,j], R[1:j-1,j]) )
    end
    triu(R)
    end

DenseCholCol1 (generic function with 1 method)

In [64]:
function DenseCholCol2(R)
    n = size(R)[1]
    for j in 1:n
        for i in 1:j-1
            # println("i ", i)
            R[i,j] = (R[i,j] - dot(R[1:i-1,i],R[1:i-1,j]))/R[i,i]  
        end
        # println("j ", j)
        R[j,j] = sqrt(R[j,j] - dot(R[1:j-1,j], R[1:j-1,j]) )
    end
    triu!(R)
    end

DenseCholCol2 (generic function with 1 method)

In [81]:
using LinearAlgebra # Add Benchmark to use it instead of time.  
m=256;A=rand(m,m);A0=A'*A; 
U=copy(A0); R2=copy(A0)
@time DenseCholRow(U)
@time R1 = DenseCholCol1(A0)
@time DenseCholCol2(R2)
map(norm,(A0-U'*U, A0-R1'*R1,A0-R2'*R2))

  0.047518 seconds (131.08 k allocations: 100.871 MiB, 13.40% gc time)
  0.011439 seconds (65.80 k allocations: 51.191 MiB, 16.54% gc time)
  0.009840 seconds (65.80 k allocations: 50.191 MiB, 18.22% gc time)


(7.574278914386446e-12, 7.098794898389419e-12, 7.098794898389419e-12)

# Sparse Cholesky Decomposition
For a Sparse SPD A a simple in-place incomplete (zero-fill) Cholesky Decompostion is 

In [4]:
import Pkg; Pkg.add("MatrixDepot")

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [b51810bb] + MatrixDepot v1.0.7
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [a74b3585] + Blosc v0.5.1
  [e1450e63] + BufferedStreams v1.0.0
  [631607c0] + CMake v1.2.0
  [d5fb7624] + CMakeWrapper v0.2.4
  [324d7699] + CategoricalArrays v0.8.3
  [a93c6f00] + DataFrames v0.21.8
  [e2d170a0] + DataValueInterfaces v1.0.0
  [f67ccb44] + HDF5 v0.12.5
  [41ab1584] + InvertedIndices v1.1.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [23992714] + MAT v0.7.0
  [b51810bb] + MatrixDepot v1.0.7
  [2dfb63ee] + PooledArrays v0.5.3
  [6c6a2e73] + Scratch v1.1.0
  [856f2bd8] + StructTypes v1.8.1
  [3783bdb8] + TableTraits v1.0.1
  [bd369af6] + Tables v1.7.0
  [9fa8497b] + Future 


In [82]:
using MatrixDepot
mdinfo()

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/allanstruthers/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:139
┌ Info: reading database
└ @ MatrixDepot /Users/allanstruthers/.julia/packages/MatrixDepot/GEDc3/src/download.jl:23
┌ Info: adding metadata...
└ @ MatrixDepot /Users/allanstruthers/.julia/packages/MatrixDepot/GEDc3/src/download.jl:67
┌ Info: adding svd data...
└ @ MatrixDepot /Users/allanstruthers/.julia/packages/MatrixDepot/GEDc3/src/download.jl:69
┌ Info: writing database
└ @ MatrixDepot /Users/allanstruthers/.julia/packages/MatrixDepot/GEDc3/src/download.jl:74
┌ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index
└ @ MatrixDepot /Users/allanstruthers/.julia/packages/MatrixDepot/GEDc3/src/MatrixDepot.jl:141


### Currently loaded Matrices

| builtin(#)  |             |              |             |               |
|:----------- |:----------- |:------------ |:----------- |:------------- |
| 1 baart     | 13 fiedler  | 25 invhilb   | 37 parter   | 49 shaw       |
| 2 binomial  | 14 forsythe | 26 invol     | 38 pascal   | 50 smallworld |
| 3 blur      | 15 foxgood  | 27 kahan     | 39 pei      | 51 spikes     |
| 4 cauchy    | 16 frank    | 28 kms       | 40 phillips | 52 toeplitz   |
| 5 chebspec  | 17 gilbert  | 29 lehmer    | 41 poisson  | 53 tridiag    |
| 6 chow      | 18 golub    | 30 lotkin    | 42 prolate  | 54 triw       |
| 7 circul    | 19 gravity  | 31 magic     | 43 randcorr | 55 ursell     |
| 8 clement   | 20 grcar    | 32 minij     | 44 rando    | 56 vand       |
| 9 companion | 21 hadamard | 33 moler     | 45 randsvd  | 57 wathen     |
| 10 deriv2   | 22 hankel   | 34 neumann   | 46 rohess   | 58 wilkinson  |
| 11 dingdong | 23 heat     | 35 oscillate | 47 rosser   | 59 wing       |
| 12 erdrey   | 24 hilb     | 36 parallax  | 48 sampling |               |

| user(#) |
|:------- |

| Groups  |       |       |         |        |         |           |     |     |     |     |     |
|:------- |:----- |:----- |:------- |:------ |:------- |:--------- |:--- |:--- |:--- |:--- |:--- |
| all     | local | eigen | illcond | posdef | regprob | symmetric |     |     |     |     |     |
| builtin | user  | graph | inverse | random | sparse  |           |     |     |     |     |     |

| Suite Sparse | of   |
|:------------ |:---- |
| 1            | 2893 |

| MatrixMarket | of  |
|:------------ |:--- |
| 0            | 498 |


In [15]:
import Pkg; Pkg.add("UnicodePlots")


 Resolving package versions...
 Installed Crayons ────── v4.0.4
 Installed UnicodePlots ─ v2.6.0
  Updating `~/.julia/environments/v1.0/Project.toml`
  [b8865327] + UnicodePlots v2.6.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [a8cc5b0e] + Crayons v4.0.4
  [b8865327] + UnicodePlots v2.6.0


In [84]:
md = mdopen("Rommes/bips07_1998")
A = md.A;


In [90]:
m=123;
ASub = A[1:m,1:m]
norm(ASub-ASub')
#eigen(Matrix(ASub)).values

1412.2033979552043

In [20]:
listgroups()

13-element Array{Symbol,1}:
 :all      
 :builtin  
 :local    
 :user     
 :eigen    
 :graph    
 :illcond  
 :inverse  
 :posdef   
 :random   
 :regprob  
 :sparse   
 :symmetric

# Planning and Thinking
It is clear that one should run through by column and update in the appropriate memory location.  A big question is what the time impact for the Upper Triangularization is. I thought I chose a symmetric matrix. Apparently I did not!

In [114]:
using LinearAlgebra
md = mdopen("Rommes/bips07_1998")
@time A = md.A
@time B = copy(A)
@time Bt= copy(A')
@time SymTest = Bt-B
@time SymTest = B'-B
norm(SymTest)

  0.038320 seconds (71.67 k allocations: 7.068 MiB)
  0.000261 seconds (11 allocations: 1.065 MiB)
  0.000441 seconds (13 allocations: 1.065 MiB)
  0.000553 seconds (11 allocations: 2.014 MiB)
  0.003867 seconds (20 allocations: 3.078 MiB, 71.70% gc time)


2.8284271247461934e12

In [158]:
(i,j)=(1412,1823)
println("Extract")
@time (ci = A[:,i]; cj = A[:,j])
println("Sparse Dot")
@time dot(ci,cj)
println("Extract and Sparse Dot")
@time dot(A[:,i],A[:,j])
println("Densify")
@time Ci=Vector(ci); Cj=Vector(cj);
println("Dense Dot")
@time dot(Ci,Cj)

Extract
  0.000012 seconds (10 allocations: 640 bytes)
Sparse Dot
  0.000008 seconds (5 allocations: 176 bytes)
Extract and Sparse Dot
  0.000012 seconds (11 allocations: 656 bytes)
Densify
  0.000031 seconds (6 allocations: 117.984 KiB)
Dense Dot
  0.000065 seconds (5 allocations: 176 bytes)


0.0

In [142]:
(ci.n,ci.nzind, ci.nzval)

(15066, [5, 12], [0.0170483, -1.0])

In [44]:
using SparseArrays
m=12;
A=sprand(m,m,0.05)+I
A=A'*A

12×12 SparseMatrixCSC{Float64,Int64} with 34 stored entries:
  [1 ,  1]  =  1.34563
  [6 ,  1]  =  0.587904
  [9 ,  1]  =  0.0842642
  [2 ,  2]  =  1.0
  [9 ,  2]  =  0.88101
  [3 ,  3]  =  1.0
  [9 ,  3]  =  0.894942
  [4 ,  4]  =  1.0
  [9 ,  4]  =  0.545167
  [12,  4]  =  0.942245
  [5 ,  5]  =  1.0
  [8 ,  5]  =  0.544351
  ⋮
  [4 ,  9]  =  0.545167
  [6 ,  9]  =  0.14333
  [9 ,  9]  =  2.89485
  [12,  9]  =  0.513681
  [7 , 10]  =  0.57439
  [10, 10]  =  1.32992
  [11, 11]  =  1.0
  [12, 11]  =  0.964448
  [4 , 12]  =  0.942245
  [9 , 12]  =  0.513681
  [11, 12]  =  0.964448
  [12, 12]  =  2.81798

In [53]:
js=A.rowval
ps=A.colptr
n=A.n
ro[ps[1:n-1] ps[2:n]] 

11×2 Array{Int64,2}:
  1   4
  4   6
  6   8
  8  11
 11  13
 13  16
 16  18
 18  20
 20  27
 27  29
 29  31

In [53]:
function SparseZeroFillIncompleteCholesky(U) 
    m = size(U)[1]
    cols = U.colptr
    is = U.rowvals
    zs = U.nzvals
    for k in 1:m
        for j in k+1:m
            U[j,j:m] -= (U[k,j]/U[k,k])*U[k,j:m] 
        end
        U[k,k:m] = U[k,k:m]/sqrt(U[k,k])
    end
    triu!(U)
    end

DenseChol (generic function with 1 method)